In [1]:
import pandas as pd
import numpy  as np

## Preprocessing

In [2]:
business = pd.read_csv('DUK.csv')
business = business[['Date', 'Adj Close']]
returns  = np.log(business['Adj Close'].values[1:]) - np.log(business['Adj Close'].values[:-1]) # price to return
business.loc[1:,'Adj Close'] = returns                                                          # eliminating first row as it was used to compute returns
business = business.iloc[1:-3,:]
business['Adj Close'] = business['Adj Close'] * 12

rf_rate  = pd.read_csv('rf_rate.csv')
rf_rate  = rf_rate[['Date', 'Adj Close']]
rf_rate  = rf_rate.iloc[1:-2,:]
rf_rate['Adj Close'] = rf_rate['Adj Close'] /100

In [3]:
market = pd.read_csv('49_industry_portfolios.csv')
market = market[['Date', 'Util ']]
market['Date'] = market.Date.apply(lambda x: str(x)[:4] + f'-{str(x)[4:6]}'+ '-01')
init_index = np.where(market.Date.values == business.Date.values[0 ])[0][0]
last_index = np.where(market.Date.values == business.Date.values[-1])[0][0]
market = market.iloc[init_index:last_index+1,:]
market['Util '] = market['Util ']/ 100 * 12

### Merging

In [4]:
data = business.merge(rf_rate, left_on='Date', right_on='Date', suffixes=('_business','_rf'))
data = data.merge(market, left_on='Date', right_on='Date', suffixes=('','_market'))

In [10]:
data = data.iloc[:190] # last month dec 2019

## Computing results

In [11]:
betas = []
get_window = lambda x,index,size: x.values[index: index + size]
for idx in range(business.shape[0] - 60):
    variance   = np.var(get_window(data['Adj Close_business'], idx, 60))
    covariance = np.cov(get_window(data['Adj Close_business'], idx, 60),get_window(data['Util '], idx, 60))[0,1]
    betas.append(covariance/variance)
betas = np.array(betas)

In [12]:
beta       = np.median(betas)
risk_free  = data['Adj Close_rf'].mean()
risk_prem  = data['Util '].mean() - risk_free
r_hydro    = risk_free + beta * risk_prem
r_real     = (1 + r_hydro) / (1 + 0.02) - 1 

print(beta)
print(risk_free)
print(risk_prem)
print(r_hydro)
print(r_real)

0.6106978053189916
0.012855105263157895
0.09239752631578947
0.0692820718011143
0.048315756667759
